In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import zipfile
import os
from io import BytesIO

# --- Configuration and Secrets ---
# Streamlit Cloud will automatically set OPENAI_API_KEY from secrets
# Ensure your key is set in Streamlit Cloud deployment settings!

# --- Utility Function to Load Data from ZIP ---
@st.cache_data
def load_data_from_zip(uploaded_file):
    """Extracts a CSV or Excel file from the uploaded ZIP and loads it into a DataFrame."""
    try:
        # Create an in-memory ZIP file object
        with zipfile.ZipFile(uploaded_file, 'r') as zip_ref:
            file_list = zip_ref.namelist()

            csv_files = [f for f in file_list if f.lower().endswith('.csv')]
            excel_files = [f for f in file_list if f.lower().endswith(('.xlsx', '.xls'))]

            if csv_files:
                # Read the first CSV file found
                with zip_ref.open(csv_files[0]) as file:
                    df = pd.read_csv(file)
                    return df, csv_files[0]
            elif excel_files:
                # Read the first Excel file found
                with zip_ref.open(excel_files[0]) as file:
                    df = pd.read_excel(file)
                    return df, excel_files[0]
            else:
                return None, None
    except Exception as e:
        st.error(f"Error processing file: {e}")
        return None, None

# --- Generative AI Function ---
def generate_insights(df, task):
    """Generates business insights using the OpenAI API."""

    # 1. Prepare Prompt Data
    head_text = df.head().to_markdown(index=False)
    summary_text = df.describe(include='all').to_markdown()

    prompt = f"""
    You are an expert business data analyst. Analyze the provided dataset and generate a concise report
    with key business insights and actionable recommendations.

    Data Head (First 5 Rows):
    {head_text}

    Data Summary (Statistics):
    {summary_text}

    Specific Request/Task: {task}

    Provide your analysis and recommendations in a professional, markdown-formatted business report.
    """

    # 2. Call OpenAI API
    try:
        # Check if the API key is set in environment variables (via Streamlit secrets)
        api_key = os.environ.get('OPENAI_API_KEY')
        if not api_key:
             return "⚠️ **OpenAI API Key Not Found.** Please set your `OPENAI_API_KEY` in the Streamlit Cloud Secrets or environment variables to enable AI insights."

        client = openai.OpenAI(api_key=api_key)

        response = client.chat.completions.create(
            model="gpt-4o-mini", # A fast and capable model
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1024,
            temperature=0.6 # Moderate creativity for balanced reports
        )
        return response.choices[0].message.content

    except Exception as e:
        return f"An error occurred during AI generation. Check your API key, model name, and rate limits: {e}"


# --- Streamlit App Layout ---
st.set_page_config(layout="wide", page_title="AI-Powered Business Insight Generator")
st.title("💡 AI-Powered Business Insight Generator")
st.markdown("Automate business reports with data analysis and Generative AI.")

# 1. Data Upload
with st.sidebar:
    st.header("1️⃣ Data Upload")
    uploaded_file = st.file_uploader(
        "Upload your ZIP file containing a single CSV or Excel file.",
        type=["zip"]
    )

    st.markdown("---")
    st.header("⚙️ AI Configuration")
    ai_task = st.text_area(
        "What specific insights/recommendations do you need?",
        "Identify top 3 revenue drivers and suggest strategies to reduce customer churn.",
        height=100
    )
    generate_button = st.button("Generate Business Report")


df = None
file_name = None

if uploaded_file is not None:
    # Load data
    df, file_name = load_data_from_zip(uploaded_file)

    if df is not None:
        st.success(f"Successfully loaded data from **{file_name}**.")
    else:
        st.error("Could not load a valid CSV or Excel file from the ZIP.")

if df is not None:

    # --- Data Display and Summary ---
    st.header("2️⃣ Data Overview")

    tab1, tab2 = st.tabs(["Data Preview", "Statistical Summary"])

    with tab1:
        st.subheader(f"Data Preview ({df.shape[0]} rows, {df.shape[1]} columns)")
        st.dataframe(df.head(10), use_container_width=True)

    with tab2:
        st.subheader("Statistical Summary")
        st.dataframe(df.describe(include='all'), use_container_width=True)

    st.markdown("---")

    # --- Data Visualization ---
    st.header("3️⃣ Key Visualizations")

    numeric_cols = df.select_dtypes(include='number').columns

    if len(numeric_cols) > 0:

        col_plot, col_select = st.columns([3, 1])

        with col_select:
            # Interactive column selector
            selected_col = st.selectbox(
                "Select a numeric column:",
                numeric_cols,
                index=0
            )

        with col_plot:
            st.subheader(f"Distribution of {selected_col}")
            # Generate plot
            fig, ax = plt.subplots(figsize=(10, 5))
            # Use .dropna() to handle missing values gracefully
            sns.histplot(df[selected_col].dropna(), kde=True, ax=ax, bins=30)
            st.pyplot(fig)

    else:
        st.info("⚠️ No numeric columns found for plotting.")

    st.markdown("---")

    # --- Generative AI Insight ---
    st.header("4️⃣ AI-Powered Report")

    if generate_button:
        with st.spinner("Analyzing data and generating report..."):
            report = generate_insights(df.copy(), ai_task)
            st.markdown(report)

else:
    # Initial message when no file is uploaded
    st.info("👆 Please upload a ZIP file containing your business data (CSV or Excel) to begin the analysis and generate the report.")

Writing app.py


In [3]:
%%writefile requirements.txt
streamlit
pandas
matplotlib
seaborn
openai

Writing requirements.txt
